In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
embeddings

In [ ]:
llm

In [ ]:
llm.invoke("What is the capital of France?").content

## Tools

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper, return_direct=True, verbose=True)

In [ ]:
print(tool.run("Who is Max Verstappen?"))

## Create your custom tool

In [ ]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Get the length of a word."""
    return len(word)


# We have to define a docstring for the tool to work properly, without it, it won't show up in the list of tools the agent can use and give error.

In [ ]:
get_word_length.invoke("Hello World")

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

In [ ]:
add_numbers.invoke({"a": 5, "b": 10})

## Concept of Agents

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [ ]:
tools = load_tools(["wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [ ]:
agent.run("Hello how are you?")

In [ ]:
agent.run("Who is the current world champion in Formula 1 racing?")

In [ ]:
agent.run("What is GPT-2?")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily basically provides a search engine tool that can be used to fetch real-time information from the web.

In [ ]:
search = TavilySearchResults()

In [ ]:
search.run("Who is the current world champion in Formula 1 racing?")

In [ ]:
search.invoke("Who is the current world champion in Formula 1 racing?")

In [ ]:
tools = [search]

In [ ]:
from langchain import hub

In [ ]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

agent1 = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    early_stopping_method="generate",   # <- if it’s stuck, ask LLM to write a final answer
)

In [ ]:
agent_executor.invoke({"input": "Who is John Wick?"})

## Self Ask with Search Agent Langgraph

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/John_Wick")
docs = loader.load()

In [ ]:
docs

In [ ]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [ ]:
vector  = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("Who is John Wick?")[0].page_content

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="JohnWickRetriever",
    description="Useful for when you need to find information about John Wick from Wikipedia.",
)

In [ ]:
tools1 = [search, retriever_tool]

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [ ]:
agent2 = create_tool_calling_agent(llm, tools1, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent2, tools=tools1, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is Johns profession?"})

In [ ]:
agent_executor.invoke({"input": "What is the weather in New York?"})

In [ ]:
search2 = TavilySearchResults(k=3)
tool2 = [search2]

In [ ]:
agent3 = create_tool_calling_agent(llm, tool2, prompt)

In [ ]:
agent_executor1 = AgentExecutor(agent=agent3, tools=tool2, verbose=True, max_iterations=4, early_stopping_method="generate")

In [ ]:
agent_executor1.invoke({"input": "What is the weather in New York as of 29th August 2025?"})

## ReAct Agent (Reasoning and Action Agent)

In [4]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [7]:
google_search = GoogleSerperAPIWrapper()

In [8]:
tools = [
    Tool(
        name = "Web Search Functionality",
        func = google_search.run,
        description = "Useful for when you need to find information about current events or topics that are not in your training data. Input should be a search query.",
        verbose = True
    )
]

In [9]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [10]:
prompt = PromptTemplate.from_template(template)

In [11]:
search_agent_react = create_react_agent(llm,tools,prompt)

In [12]:
agent_executor = AgentExecutor(
    agent = search_agent_react,
    tools = tools,
    verbose = True,
    return_intermediate_steps=True
)

In [13]:
agent_executor.invoke({"input": "Who won the latest football world cup?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I need to find out who won the most recent football (soccer) World Cup. This is a specific event that has a definitive answer, but I should use the web search tool to ensure I have the most current information.
Action: Web Search Functionality
Action Input: "who won the latest football world cup"The reigning champions are Argentina, who won their third title at the 2022 World Cup by defeating France. FIFA World Cup. Organising body, FIFA. Founded ... Argentina won the most recent World Cup in 2022, defeating France in a thrilling final. Brazil holds the record with 5 World Cup championships. 8 different ... France started their World Cup campaign as the defending world champions, having won the most recent tournament in Russia, and was drawn in group D. ... With the ... AWARDS ; 2022. Argentina Flag · Argentina ; 2018. France Flag · France ; 2014. Germany Flag · Germany ; 2010. Spain Flag · Spain ; 2006. Italy Flag · Italy. The Moment When Argentina Won The 2022 FIFA World Cup

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I now know the final answer
Final Answer: Argentina won the latest football World Cup, which was the 2022 FIFA World Cup. They defeated France in the final.

> Finished chain.


{'input': 'Who won the latest football world cup?',
 'output': 'Argentina won the latest football World Cup, which was the 2022 FIFA World Cup. They defeated France in the final.',
 'intermediate_steps': [(AgentAction(tool='Web Search Functionality', tool_input='who won the latest football world cup', log='Thought: I need to find out who won the most recent football (soccer) World Cup. This is a specific event that has a definitive answer, but I should use the web search tool to ensure I have the most current information.\nAction: Web Search Functionality\nAction Input: "who won the latest football world cup"'),
   "The reigning champions are Argentina, who won their third title at the 2022 World Cup by defeating France. FIFA World Cup. Organising body, FIFA. Founded ... Argentina won the most recent World Cup in 2022, defeating France in a thrilling final. Brazil holds the record with 5 World Cup championships. 8 different ... France started their World Cup campaign as the defending w